In [105]:
%matplotlib widget

In [106]:
import numpy as np
import matplotlib.pyplot as plt

In [108]:
from rl_2sources import rl_2sources_class 

## Instantiate system

In [109]:
syst = rl_2sources_class()

## Solve steady state

In [110]:
syst.initialize()

True

In [111]:
syst.report_x()
syst.report_y()
syst.report_u()

i     = 10.00
p     = 1000.00
v_1   = 101.00


## Run simulation

In [125]:
events = [{'p':1000.0},
          {'t_end':0.1,'v_1':102},
          {'t_end':1.0,'v_1':102},
          {'t_end':2.0,'v_1':100}]
syst.simulate(events);

In [126]:
fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(7, 7))

axes[0].plot(syst.T,syst.get_values('i'))
axes[1].plot(syst.T,syst.get_values('p'))


for ax in axes:
    ax.grid()
    #ax.legend()
ax.set_xlabel('Time (s)')


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0, 'Time (s)')

## Run interactive simulation

In [130]:
Δt = 0.01 
times = np.arange(0.0,2,Δt)

In [131]:
syst.initialize()

it = 0
for t in times:
    
    v_1 = 101 + np.sin(2*np.pi*1.0*t)
           
    events=[{'t_end':t,'v_1':v_1}]
    syst.run(events)

    it += 1
    
syst.post();

In [132]:
fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(7, 7))

axes[0].plot(syst.T,syst.get_values('i'))
axes[1].plot(syst.T,syst.get_values('v_1'))


for ax in axes:
    ax.grid()
    #ax.legend()
ax.set_xlabel('Time (s)')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0, 'Time (s)')

In [138]:
syst.Dt

0.001

## Run simulation with feedback control (proportional control)

In [135]:
Δt = 0.1 
times = np.arange(0.0,2,Δt)

In [139]:
syst.initialize()
K_p = 2e-3
K_i = 0.01
syst.xi = 0.0
it = 0
for t in times:
    
    p = syst.get_value('p')
    
    p_ref = 1000
    
    if t>0.1:
        p_ref = 1200
    
    error = (p_ref - p)
    
    
    v_1 = 101 + K_p*error + K_i*syst.xi
    
    syst.xi += Δt*error
           
    events=[{'t_end':t,'v_1':v_1}]
    syst.run(events)

    it += 1
    
syst.post();

In [140]:
fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(7, 7))

axes[0].plot(syst.T,syst.get_values('p'))
axes[1].plot(syst.T,syst.get_values('v_1'))


for ax in axes:
    ax.grid()
    #ax.legend()
ax.set_xlabel('Time (s)')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0, 'Time (s)')

In [15]:
T_ctrl = np.zeros((len(times),1))
X_ctrl = np.zeros((len(times),N_u_d*2))
U_ctrl = np.zeros((len(times),2))
X_obs = np.zeros((len(times),N_x_d))
Z_obs = np.zeros((len(times),N_z_o))

    z_ref = np.copy(np.array([[p_t_ref],
                              [v_1_ref]]))
    
    u_d = control(syst,z_ref)
    
    p_m_ref = u_d[0,0]
    v_f =     u_d[1,0]

IndentationError: unexpected indent (<ipython-input-15-69b5124a0a58>, line 7)

## Run simulation with feedback control (PI control)

In [ ]:
Δt = 0.1 
times = np.arange(0.0,2,Δt)

In [ ]:
syst.initialize()
K_p = 2e-3
K_i = 0.01
syst.xi = 0.0
it = 0
for t in times:
    
    p = syst.get_value('p')
    
    p_ref = 1000
    
    if t>0.1:
        p_ref = 1200
    
    error = (p_ref - p)
    
    
    v_1 = 101 + K_p*error + K_i*syst.xi
    
    syst.xi += Δt*error
           
    events=[{'t_end':t,'v_1':v_1}]
    syst.run(events)

    it += 1
    
syst.post();

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(7, 7))

axes[0].plot(syst.T,syst.get_values('p'))
axes[1].plot(syst.T,syst.get_values('v_1'))


for ax in axes:
    ax.grid()
    #ax.legend()
ax.set_xlabel('Time (s)')

## PI control design

In [141]:
import pydae.ssa as ssa
import scipy.signal as sctrl


### Sistema DAE linealizado
\begin{eqnarray}
\Delta \dot x_p &=& A_p \Delta x_p + B_p \Delta u_p \\
\Delta z_p &=& C_p \Delta x_p + D_p \Delta u_p \\
\end{eqnarray}

### Discretized system
\begin{eqnarray}
\Delta x_d^{k+1} &=& A_d \Delta x_d^{k}  + B_d \Delta u_d^{k}  \\
\Delta z_d^{k} &=& C_d^{k} \Delta x_d^{k} + D_d^{k} \Delta u_d^{k} \\
\end{eqnarray}

### Dynamic extension
\begin{eqnarray}
\Delta x_i^{k+1} &=& \Delta x_i^{k} + \Delta t \left(\Delta z_c- \Delta z_c^\star \right)  \\
\end{eqnarray}

with: 
\begin{eqnarray}
\Delta z_c  &=& z_c - z_c^0  \\
\Delta z_c^\star &=& z_c^\star -z_c^0 
\end{eqnarray}

### Extended system
\begin{equation}
\Delta x_e^{k+1}  = 
\begin{bmatrix}
A_d & 0\\
\Delta t C_c & I
\end{bmatrix}
\begin{bmatrix}
\Delta x_d^{k}\\
\Delta x_i^{k}
\end{bmatrix}
+
\begin{bmatrix}
B_d \\
0
\end{bmatrix}
\Delta u_d^{k} 
\end{equation}
 
\begin{equation}
A_e  = 
\begin{bmatrix}
A_d & 0\\
\Delta t C_c & I
\end{bmatrix}
\;\;\;\;\;
B_e = 
\begin{bmatrix}
B_d \\
0
\end{bmatrix}
\end{equation}


### State feedback control
\begin{equation}
\Delta u_d^{k} = -K_e \Delta x_e^{k}
\end{equation}


$K_e$ can be obtained with LQR:

K_e = lqr(A_e,B_e,Q,R)

In [144]:
Δt = 0.1 
x_d_ctrl_list = ['i']   # states to consider in the reduction
z_ctrl_list = ['p']     # outputs to consider in the controller

z_ctrl_idxs = [syst.outputs_list.index(item) for item in z_ctrl_list]

syst.Δt = Δt

## Calculate equilibirum point
syst.initialize([{'p':1e3}])
ssa.eval_ss(syst)

# linear continous plant
A_p = syst.A
B_p = syst.B
C_p = syst.C
D_p = syst.D

# plant discretization
A_d,B_d,C_d,D_d,Dt = sctrl.cont2discrete((A_p,B_p,C_p,D_p),Δt,method='zoh')

N_z_d,N_x_d = C_d.shape  # discreticed plant dimensions
N_x_d,N_u_d = B_d.shape

# convenient matrices
O_ux = np.zeros((N_u_d,N_x_d))
O_xu = np.zeros((N_x_d,N_u_d))
O_uu = np.zeros((N_u_d,N_u_d))
I_uu = np.eye(N_u_d)

syst.A_d = A_d
syst.B_d = B_d


# Controller ##################################################################################
C_c = C_d[z_ctrl_idxs,:]
D_c = D_d[z_ctrl_idxs,:]

N_z_c,N_x_c = C_c.shape

O_ux = np.zeros((N_u_d,N_x_d))
O_xu = np.zeros((N_x_d,N_u_d))
O_uu = np.zeros((N_u_d,N_u_d))
I_uu = np.eye(N_u_d)


# discretized plant:
# Δx_d = A_d*Δx_d + B_d*Δu_d
# Δz_c = C_c*Δx_d + D_c*Δu_d

# dinamic extension:
# Δx_d = A_d*Δx_d + B_d*Δu_d
# Δx_i = Δx_i + Δt*(Δz_c-Δz_c_ref) = Δx_i + Δt*C_c*Δx_d - Dt*Δz_c_ref
# Δz_c = z_c - z_c_0
# Δz_c_ref = z_c_ref - z_c_0
# (Δz_c-Δz_c_ref) = z_c - z_c_ref

A_e = np.block([
                [    A_d, O_xu],    # Δx_d
                [ Δt*C_c, I_uu],    # Δx_i    
               ])

B_e = np.block([
                [   B_d],
                [   O_uu],    
               ])



# weighting matrices
Q_c = np.eye(A_e.shape[0])
R_c = np.diag([0.1]) 

K_c,S_c,E_c = ssa.dlqr(A_e,B_e,Q_c,R_c)
E_c = np.log(E_c)/Δt

In [145]:
E_c

array([-68.85462939+0.j, -47.29891607+0.j])

## PI control validation

In [146]:
times = np.arange(0.0,2,Δt)

In [147]:
syst.initialize()
i_0 = syst.get_value('i')
K_p = K_c[0,0]
K_i = K_c[0,1]
syst.Δxi = 0.0
u_d_0 = syst.get_value('v_1')
p_0 =  syst.get_value('p')
it = 0
for t in times:
    

    p = syst.get_value('p')
    i = syst.get_value('i')
    
    Δx_d = i - i_0
    Δx_i = syst.Δxi
    
    p_ref = p_0
    
    if t>0.1:
        p_ref = 1200
    
    error = (p - p_ref)
    
    Δx_e = np.block([Δx_d, Δx_i]).T
    
    Δu_d = -K_c @ Δx_e
    
    u_d = Δu_d + u_d_0
    
    syst.Δxi += Δt*error
    
    v_1 = u_d
    
    events=[{'t_end':t,'v_1':v_1}]
    syst.run(events)

    it += 1
    
syst.post();

In [148]:
fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(7, 7))

axes[0].plot(syst.T,syst.get_values('p'))
axes[1].plot(syst.T,syst.get_values('v_1'))


for ax in axes:
    ax.grid()
    #ax.legend()
ax.set_xlabel('Time (s)')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0, 'Time (s)')

In [22]:
syst.initialize()
K_p = 2e-3
it = 0
for t in times:
    
    p = syst.get_value('p')
    
    p_ref = 1000
    
    if t>0.1:
        p_ref = 1200
    
    v_1 = 101 + K_p*(p_ref - p)
           
    events=[{'t_end':t,'v_1':v_1}]
    syst.run(events)

    it += 1
    
syst.post();

## PI control with delay design

In [15]:
import pydae.ssa as ssa
import scipy.signal as sctrl


### Linealized DAE system
\begin{eqnarray}
\Delta \dot x_p &=& A_p \Delta x_p + B_p \Delta u_p \\
\Delta z_p &=& C_p \Delta x_p + D_p \Delta u_p \\
\end{eqnarray}

### Discretized system
\begin{eqnarray}
\Delta x_d^{k+1} &=& A_d \Delta x_d^{k}  + B_d \Delta u_d^{k}  \\
\Delta z_d^{k} &=& C_d  \Delta x_d^{k} + D_d  \Delta u_d^{k} \\
\end{eqnarray}

### Discretized system from control point of view
\begin{eqnarray}
\Delta x_d^{k+1} &=& A_d \Delta x_d^{k} + B_d \Delta x_r^{k}  \\
\Delta z_c^{k}   &=& C_c \Delta x_d^{k} + D_c \Delta x_r^{k} \\
\end{eqnarray}


### Delay extension
\begin{eqnarray}
\Delta x_r^{k+1} &=& \Delta u_r^{k} \\
\end{eqnarray}


### Dynamic extension
\begin{eqnarray}
\Delta x_i^{k+1} &=& \Delta x_i^{k} + \Delta t \left(\Delta z_c- \Delta z_c^\star \right)  \\
\end{eqnarray}

with: 
\begin{eqnarray}
\Delta z_c  &=& z_c - z_c^0  \\
\Delta z_c^\star &=& z_c^\star -z_c^0 \\
\Delta z_c  &=& C_c \Delta x_d^{k} + D_c \Delta u_d^{k}  
\end{eqnarray}

### Extended system

\begin{eqnarray}
\Delta x_d^{k+1} &=& A_d \Delta x_d^{k}  + B_d \Delta x_r^{k}  \\
\Delta x_r^{k+1} &=& \Delta u_r^{k} \\
\Delta x_i^{k+1} &=& \Delta x_i^{k} + \Delta t C_c \Delta x_d^k - \Delta t \Delta z_c^\star   \\
\end{eqnarray}



\begin{equation}
\Delta x_e^{k+1}  = 
\begin{bmatrix}
A_d & B_d & 0 \\
0 & 0 & 0 \\
\Delta t C_c& \Delta t D_c & I 
\end{bmatrix}
\begin{bmatrix}
\Delta x_d^{k}\\
\Delta x_r^{k}\\
\Delta x_i^{k}
\end{bmatrix}
+
\begin{bmatrix}
0\\
I \\
0
\end{bmatrix}
\Delta u_r^{k} 
\end{equation}
 
\begin{equation}
A_e  = 
\begin{bmatrix}
A_d & B_d & 0\\
0 & 0 & 0\\
\Delta t C_c & 0 & I
\end{bmatrix}
\;\;\;\;\;
B_e = 
\begin{bmatrix}
0 \\
I \\
0
\end{bmatrix}
\end{equation}


### State feedback control
\begin{equation}
\Delta u_r^{k} = -K_e \Delta x_e^{k}
\end{equation}


$K_e$ can be obtained with LQR:

K_e = lqr(A_e,B_e,Q,R)

### Control dynamics
\begin{equation}
\Delta x_{ctrl}^{k+1}  = 
\begin{bmatrix}
0 & 0 \\
0 & I 
\end{bmatrix}
\begin{bmatrix}
\Delta x_r^{k}\\
\Delta x_i^{k}
\end{bmatrix}
+
\begin{bmatrix}
I \\
0
\end{bmatrix}
\Delta u_d^{k} 
\end{equation}
 
\begin{equation}
A_{ctrl}  = 
\begin{bmatrix}
 0 & 0\\
 0 & I
\end{bmatrix}
\;\;\;\;\;
B_{ctrl} = 
\begin{bmatrix}
I \\
0
\end{bmatrix}
\end{equation}


In [159]:
Δt = 0.01
x_d_ctrl_list = ['i']   # states to consider in the reduction
z_ctrl_list = ['p']     # outputs to consider in the controller

z_ctrl_idxs = [syst.outputs_list.index(item) for item in z_ctrl_list]

syst.Δt = Δt

## Calculate equilibirum point
syst.initialize([{'p':1e3}])
ssa.eval_ss(syst)

# linear continous plant
A_p = syst.A
B_p = syst.B
C_p = syst.C
D_p = syst.D

# plant discretization
A_d,B_d,C_d,D_d,Dt = sctrl.cont2discrete((A_p,B_p,C_p,D_p),Δt,method='zoh')

N_z_d,N_x_d = C_d.shape  # discreticed plant dimensions
N_x_d,N_u_d = B_d.shape

# convenient matrices
O_ux = np.zeros((N_u_d,N_x_d))
O_xu = np.zeros((N_x_d,N_u_d))
O_uu = np.zeros((N_u_d,N_u_d))
I_uu = np.eye(N_u_d)

syst.A_d = A_d
syst.B_d = B_d


# Controller ##################################################################################
C_c = C_d[z_ctrl_idxs,:]
D_c = D_d[z_ctrl_idxs,:]

N_z_c,N_x_c = C_c.shape

O_ux = np.zeros((N_u_d,N_x_d))
O_xu = np.zeros((N_x_d,N_u_d))
O_uu = np.zeros((N_u_d,N_u_d))
I_uu = np.eye(N_u_d)


# discretized plant:
# Δx_d = A_d*Δx_d + B_d*Δu_d
# Δz_c = C_c*Δx_d + D_c*Δu_d

# dinamic extension:
# Δx_d = A_d*Δx_d + B_d*Δu_d
# Δx_i = Δx_i + Δt*(Δz_c-Δz_c_ref) = Δx_i + Δt*C_c*Δx_d - Dt*Δz_c_ref
# Δz_c = z_c - z_c_0
# Δz_c_ref = z_c_ref - z_c_0
# (Δz_c-Δz_c_ref) = z_c - z_c_ref

A_e = np.block([
                [    A_d,    B_d, O_xu],    # Δx_d
                [   O_ux,   O_uu, O_uu],    # Δx_r
                [ Δt*C_c, Δt*D_c, I_uu],    # Δx_i    
               ])

B_e = np.block([
                [   O_xu],
                [   I_uu],
                [   O_uu],    
               ])

A_ctrl = A_e[N_x_d:,N_x_d:]
B_ctrl = B_e[N_x_d:]

# weighting matrices
Q_c = np.eye(A_e.shape[0]) 
R_c = np.diag([1]) 

K_c,S_c,E_c = ssa.dlqr(A_e,B_e,Q_c,R_c)
E_c = np.log(E_c)/Δt

syst.A_ctrl = A_ctrl
syst.B_ctrl = B_ctrl
syst.K_c = K_c
syst.N_x_d = N_x_d  # number of plant states
syst.N_u_d = N_u_d  # number of plant inputs
syst.N_z_c = N_z_c  # number of plant outputs considered for the controller

In [161]:
K_c = np.array([[0.23551695, 1.35717384, 0.01565326]])

## PI control validation

In [162]:
times = np.arange(0.0,10,Δt)

In [163]:
syst.initialize()
x_0 = syst.get_x()
syst.Δx_ctrl = np.zeros((N_u_d+N_z_c,1))
syst.Δu_d = np.zeros((N_u_d,1)) 
u_d_0 = syst.get_value('v_1')
x_r_0 = u_d_0
p_0 =  syst.get_value('p')
it = 0
for t in times:
    
    p = syst.get_value('p')
    Δx_d = syst.get_x() - x_0

    
    Δx_ctrl = np.copy(syst.Δx_ctrl)  
    Δx_r = Δx_ctrl[:N_u_d]
    Δx_i = Δx_ctrl[N_u_d:]
    Δu_d = np.copy(syst.Δu_d)  
    p_ref = p_0
    
    if t>=0.1:
        p_ref = 1200
    
    Δz_c = np.block([[p - p_ref]])
    
    Δx_e = np.block([[Δx_d], [Δx_ctrl]])
    

    
    #x_r += Δu_d
    #Δx_i += Δt*Δz
    #Δx_ctrl = np.block([Δx_r, Δx_i]).T

    # control dynamics
    Δu_d = -K_c @ Δx_e
    Δx_r_k1 = Δu_d
    Δx_i_k1 = Δx_i +  Δt*Δz_c   

    
    
    #u_d = Δu_d + u_d_0
    
    syst.Δx_ctrl[:syst.N_u,:] = np.copy(Δx_r_k1)
    syst.Δx_ctrl[syst.N_u:,:] = np.copy(Δx_i_k1)    

    syst.Δu_d  = np.copy(Δu_d)     
    #yst.Δx_ctrl = np.copy(Δx_ctrl)
    
    v_1 =  Δx_r + x_r_0
   # v_1 = u_d
    events=[{'t_end':t,'v_1':v_1}]
    syst.run(events)

    it += 1
    
syst.post();

In [164]:
B_d

array([[0.95162582]])

In [165]:
fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(7, 7))

axes[0].plot(syst.T,syst.get_values('p'))
axes[1].plot(syst.T,syst.get_values('v_1'))


for ax in axes:
    ax.grid()
    #ax.legend()
ax.set_xlabel('Time (s)')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0, 'Time (s)')

In [51]:
Δt

0.01

In [102]:
Δx_e = np.zeros((N_x_d+N_u_d+N_z_c,1))
syst.Δx_ctrl = np.zeros((N_u_d+N_z_c,1))
syst.Δu_d = np.zeros((N_u_d,1)) 
u_d_0 = syst.get_value('v_1')
x_r_0 = u_d_0
p_0 =  1
X_e = np.zeros((len(times),Δx_e.shape[0]))
it = 0
for t in times:
    X_e[it,:] = Δx_e.T

    p_ref = p_0 
    if t>=0.1:
        p_ref = 1.2
    
    C_e = np.block([[C_c , 0 , 0]])
    Δz_c = np.block([[C_e@Δx_e - (p_ref-p_0)]])
    B_z = np.block([[0],[0],[-Δt]])
    
    Δu_r = -K_c @Δx_e
    Δx_e = A_e@Δx_e + B_e@Δu_r + B_z@Δz_c

    
    

    it += 1
    
syst.post();

In [103]:
C_e

array([[100.,   0.,   0.]])

In [104]:
fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(7, 7))

axes[0].plot(times,X_e)




for ax in axes:
    ax.grid()
    #ax.legend()
ax.set_xlabel('Time (s)')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0, 'Time (s)')

In [248]:
Ts_control = 0.1
Δt = Ts_control
x_d_ctrl_list = ['i']   # states to consider in the reduction
z_ctrl_list = ['p']                           # outputs to consider in the controller
z_obs_list =  ['p']       # outputs to consider in the observer

syst.Δt = Δt

## Calculate equilibirum point
events=[{'p':1000}]
syst.initialize(events,xy0=1)
print(syst.get_value('p'))
ssa.eval_ss(syst)

z_ctrl_idxs = [syst.outputs_list.index(item) for item in z_ctrl_list]
z_obs_idxs  = [syst.outputs_list.index(item) for item in z_obs_list]

# linear continous plant
A_p = syst.A
B_p = syst.B
C_p = syst.C
D_p = syst.D

N_z_p,N_x_p = C_p.shape
N_x_p,N_u_p = B_p.shape

x_ctrl_keep_idx = [syst.x_list.index(item)  for item in x_d_ctrl_list]
x_ctrl_elim_idx = list(set(range(N_x_p)) - set(x_ctrl_keep_idx)) 

# Reduction and discretization ##################################################################
if len(x_ctrl_elim_idx) == 0:
    # without reduction:
    A_d,B_d,C_d,D_d,Dt = sctrl.cont2discrete((A_p,B_p,C_p,D_p),Ts_control,method='zoh')
    T_r = np.eye(N_x_p)

else:
    # with reduction:
    sys = ctrl.ss(A_p, B_p, C_p, D_p)
    T_r =np.eye(N_x_p)
    
    rsys = ctrl.modred(sys, x_ctrl_elim_idx, method='truncate')
    A_pr,B_pr,C_pr,D_pr = rsys.A,rsys.B,rsys.C,rsys.D
    A_d,B_d,C_d,D_d,Dt = sctrl.cont2discrete((A_pr,B_pr,C_pr,D_pr),Ts_control,method='zoh')
    T_r = np.delete(np.eye(N_x_p),x_ctrl_elim_idx,axis=0)
    
N_z_d,N_x_d = C_d.shape
N_x_d,N_u_d = B_d.shape

syst.T_r = T_r  # Full linear discrete states to reduced (truncated) states 

syst.A_d = A_d
syst.B_d = B_d


# Controller ##################################################################################
C_c = C_d[z_ctrl_idxs,:]
D_c = D_d[z_ctrl_idxs,:]

N_z_c,N_x_c = C_c.shape

O_ux = np.zeros((N_u_d,N_x_d))
O_xu = np.zeros((N_x_d,N_u_d))
O_uu = np.zeros((N_u_d,N_u_d))
I_uu = np.eye(N_u_d)

# discretized plant:
# Δx_d = A_d*Δx_d + B_d*Δu_d
# Δz_c = C_c*Δx_d + D_c*Δu_d

# delay in the input:
# Δx_d = A_d*Δx_d + B_d*Δx_r
# Δz_c = C_c*Δx_c 
# Δx_r = Δu_d

# dinamic extension:
# Δx_d = A_d*Δx_d + B_d*Δx_r
# Δx_r = Δu_d
# Δx_i = Δx_i + Δt*(Δz_c-Δz_c_ref) = Δx_i + Δt*C_c*Δx_d - Dt*Δz_c_ref
# Δz_c = z_c - z_c_0
# Δz_c_ref = z_c_ref - z_c_0
# (Δz_c-Δz_c_ref) = z_c - z_c_ref

A_e = np.block([
                [    A_d,  B_d, O_xu],    # Δx_d
                [   O_ux, O_uu, O_uu],    # Δx_r
                [ Δt*C_c, O_uu, I_uu],    # Δx_i    
               ])

B_e = np.block([
                [   O_xu],
                [   I_uu],
                [   O_uu],    
               ])

# weighting matrices
Q_c = np.eye(A_e.shape[0])


R_c = np.diag([1]) 

K_c,S_c,E_c = ssa.dlqr(A_e,B_e,Q_c,R_c)
E_c = np.log(E_c)/Δt

syst.x_ctrl_keep_idx = x_ctrl_keep_idx
syst.x_ctrl_elim_idx = x_ctrl_elim_idx
syst.z_ctrl_list = z_ctrl_list

syst.N_z_c = N_z_c
syst.N_u_d = N_u_d

syst.x_ctrl = np.zeros((N_u_d+N_z_d,1))
syst.K_c = K_c
syst.C_c = C_c
syst.D_c = D_c

# Observer ###########################################################

# discretized plant:
# Dx_d = A_d*Dx_d + B_d*Du_d
# z_o  = C_o*Dx_d + D_o*Du_d

# x_o = A_d*x_o + B_d*u_d + L_o*(z_o - C_o*x_o - D_o*Du_d)

C_o = C_d[z_obs_idxs,:]
D_o = D_d[z_obs_idxs,:]

N_z_o = C_o.shape[0]

Q_o = np.eye(A_d.shape[0])


R_o = np.diag([1]*N_z_o)
K_o_T,S_o,E_o = ssa.dlqr(A_d.T,C_o.T,Q_o,R_o)
K_o = K_o_T.T

syst.K_o = K_o
syst.C_o = C_o
syst.D_o = D_o

syst.z_obs_list = z_obs_list
syst.N_z_o = N_z_o
syst.x_obs = np.zeros((N_x_d,1))

print('damp_ctrl',-E_c.real/np.abs(E_c))
print('damp_obs',-E_o.real/np.abs(E_o))

1000.0000000000063
damp_ctrl [0.96560365 0.96560365 0.99270532]
damp_obs [-1.]


In [249]:
E_c

array([ -43.10502553+11.6073527j ,  -43.10502553-11.6073527j ,
       -258.66973143+31.41592654j])

In [261]:
def control(syst,z_ref):
    
    Δt = syst.Δt                    # disctretization time
    T_r = syst.T_r                  # Full linear discrete states to reduced (truncated) states
    
    x_d = T_r @ syst.struct[0].x    # plant dynamic states (discrete)
    x_d_0 = syst.x_d_0              # plant initial steady state (discrete)
    Δx_d = x_d - x_d_0              # plant dynamic states increment (discrete)
    
    x_r_0  = syst.x_r_0              # initial delays values
    
    Δx_e = np.copy(syst.Δx_e)     # extended plant dynamic states increment (discrete)

    u_d = np.copy(syst.u_d)       # plant inputs
    u_d_0 = syst.u_d_0              # plant initial steady inputs (discrete)
    Δu_d = syst.Δu_d              # plant inputs increment (discrete)

    z_c_0 = np.copy(syst.z_c_0)     # controller considered initial plant ouputs
    z_o_0 = np.copy(syst.z_o_0)     # observer considered initial plant ouputs
    
    A_d = syst.A_d            # plant A matrix without delay and integrators extension (discrete)
    B_d = syst.B_d            # plant B matrix without delay and integrators extension (discrete)
    C_c = syst.C_c            # controller considered plant C matrix (discrete)
    D_c = syst.D_c            # controller considered plant D matrix (discrete) 
    C_o = syst.C_o            # observer considered plant C matrix (discrete) 
    D_o = syst.D_o            # observer considered plant D matrix (discrete)   
    
    N_x_d,N_u_d = B_d.shape  # dimensions
    N_z_c,N_x_c = C_c.shape
    N_z_o,N_x_o = C_o.shape    
    
    K_c = syst.K_c            # controller gains matrix
    K_o = syst.K_o            # observer gains matrix
    
    Δx_c = np.copy(syst.x_ctrl)  # controler states
    Δx_r = Δx_c[:syst.N_u_d,:]   # delay discrete states
    Δx_i = Δx_c[syst.N_u_d:,:]   # integrators discrete states
    
    Δx_o = syst.x_obs       # observer states

     
    # outputs to control         
    z_ctrl_values_list = []
    for item in syst.z_ctrl_list:
         z_ctrl_values_list += [syst.get_value(item)]
    z_c = np.array(z_ctrl_values_list).reshape(N_z_c,1)
    
    Δz_c =   (z_c-z_c_0) - (z_ref-z_c_0)
    
    
    # outputs for observer         
    z_obs_values_list = []
    for item in syst.z_obs_list:
         z_obs_values_list += [syst.get_value(item)]
    z_o = np.array(z_obs_values_list).reshape(N_z_o,1)
    
    Δz_o = z_o - z_o_0
    

    # control dynamics
    Δx_r_k1 = Δu_d
    Δx_i_k1 = Δx_i + Δt*Δz_c   

    
    # control law  
    Δu_d = -K_c @ Δx_e 

    # observer dynamics
    Δx_o_m1 = A_d @ Δx_o + B_d@Δu_d  +  K_o @ (Δz_o - C_o @ Δx_o - D_o @ Δu_d)
    
    # save statates for next step
    syst.x_ctrl[:syst.N_u,:] = np.copy(Δx_r_k1)
    syst.x_ctrl[syst.N_u:,:] = np.copy(Δx_i_k1)
    syst.x_obs = np.copy(Δx_o_m1)
    
    if syst.observer == False:
        syst.Δx_e  = np.copy(np.block([[Δx_d],[Δx_r_k1],[Δx_i_k1]]))
        
    if syst.observer == True:    
        syst.Δx_e  = np.copy(np.block([[Δx_o_m1],[Δx_r_k1],[Δx_i_k1]]))
        
    syst.Δu_d = Δu_d
    
    # outputs for post-processing
    syst.z_obs = C_o @ Δx_o + D_o @ Δu_d
       
    return Δx_r_k1 + x_r_0

In [265]:
B_d

array([[6.32120559]])

## Initialization

In [262]:
times = np.arange(0.0,20,Δt)

## Calculate initial references
events=[{'p':1000}]
syst.initialize(events,xy0=1.0)


# initial inputs
u_values_list = []
for item in syst.u_run_list:
     u_values_list += [syst.get_value(item)]
u_d_0 = np.array(u_values_list).reshape(syst.N_u_d,1)

# initial states
x_d_0 = T_r @ syst.struct[0].x    # initial plant states
x_r_0 = u_d_0                     # initial delay states
x_i_0 = np.zeros((syst.N_u_d,1))  # initial integrator states


Δx_e = np.block([[x_d_0*0],[x_r_0*0],[x_i_0*0]])

# initial outputs
z_ctrl_values_list = []
for item in syst.z_ctrl_list:
     z_ctrl_values_list += [syst.get_value(item)]
z_c_0 = np.array(z_ctrl_values_list).reshape(N_z_c,1)

# outputs for observer         
z_obs_values_list = []
for item in syst.z_obs_list:
     z_obs_values_list += [syst.get_value(item)]
z_o_0 = np.array(z_obs_values_list).reshape(N_z_o,1)

syst.Δx_e = Δx_e
syst.Δu_d = np.zeros((syst.N_u_d,1))
syst.x_d_0 = x_d_0
syst.x_r_0 = x_r_0
syst.u_d_0 = u_d_0
syst.z_c_0 = z_c_0
syst.z_o_0 = z_o_0

syst.u_d =  np.copy(u_d_0)
syst.x_ctrl = np.block([[x_r_0],[x_i_0]])*0
syst.x_obs = np.copy(x_d_0)*0

syst.observer = False

## Simulation 

In [263]:
T_ctrl = np.zeros((len(times),1))
X_ctrl = np.zeros((len(times),N_u_d*2))
U_ctrl = np.zeros((len(times),2))
X_obs = np.zeros((len(times),N_x_d))
Z_obs = np.zeros((len(times),N_z_o))
it = 0
for t in times:
    
    # references
    p_ref = 1000
   
    if t>=1:
        p_ref = 1200

        
    z_ref = np.copy(np.array([[p_ref],
                              ]))
    
    u_d = control(syst,z_ref)
    
    v_1 = u_d[0,0]
           
    events=[{'t_end':t,'v_1':v_1}]
    syst.run(events)
    
    #X_ctrl[it,:] = np.hstack((x_i.T))
    U_ctrl[it,:] = u_d.T
    X_ctrl[it,:] = syst.x_ctrl.T
    X_obs[it,:]  = syst.x_obs.T
    Z_obs[it,:]  = syst.z_obs.T
    it += 1
syst.post();

## Post processing

In [264]:
plt.close('all')
fig, axes = plt.subplots(nrows=2,ncols=2, figsize=(10, 5), frameon=False, dpi=80)

#axes[0,0].plot(syst.T, syst.get_values('omega'), label=f'$\omega$')
#axes[0,0].step(times,X_obs[:,1] + x_d_0[1],   label=f'$\hat \omega$')

axes[0,0].plot(syst.T, syst.get_values('p'), label=f"$p$")



axes[1,0].plot(syst.T, syst.get_values('v_1'), label=f"$v_1$")
for ax in axes.flatten():
    ax.legend(loc='best')
    ax.grid(True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

No handles with labels found to put in legend.
No handles with labels found to put in legend.
